In [1]:
import tweepy
import csv
import time
import pandas as pd
import datetime
import os
import json
from pandas.tseries.holiday import nearest_workday, \
    AbstractHolidayCalendar, Holiday, \
    USMartinLutherKingJr, USPresidentsDay, GoodFriday, \
    USMemorialDay, USLaborDay, USThanksgivingDay
from datetime import date

In [19]:
stock_information = pd.read_excel("./../../data/stock_information.xlsx")

In [20]:
stock_information["close_yesterday"] = stock_information["close"].shift(1)
#stock_information = stock_information[stock_information.date != "2021-10-11"]
stock_information["dependent"] = (stock_information["close"] - stock_information["close_yesterday"])/stock_information["close_yesterday"]
display(stock_information)


symbol       date       open       high        low      close  volume  \
0     VEGN 2021-10-11  40.020000  40.161999  39.759998  39.759998    5000   
1     VEGN 2021-10-12  39.840000  39.840000  39.669998  39.724998    4900   
2     VEGN 2021-10-13  40.070000  40.099998  39.810001  39.924000   16000   
3     VEGN 2021-10-14  40.340000  40.765999  40.340000  40.765999    3100   
4     VEGN 2021-10-15  41.060001  41.150002  41.060001  41.122002    1700   
..     ...        ...        ...        ...        ...        ...     ...   
248   VEGN 2022-10-05  31.469999  31.628000  31.120001  31.628000    1800   
249   VEGN 2022-10-06  31.559999  31.600000  31.309999  31.309999    1200   
250   VEGN 2022-10-07  30.299999  30.299999  30.180000  30.239000    1300   
251   VEGN 2022-10-10  30.350000  30.350000  29.629999  29.830000    4500   
252   VEGN 2022-10-11  29.760000  29.860001  29.540001  29.540001    3200   

      adjusted  close_yesterday  dependent  
0    39.524410              NaN        NaN  
1    39.489620        39.759998  -0.000880  
2    39.687439        39.724998   0.005009  
3    40.524448        39.924000   0.021090  
4    40.878345        40.765999   0.008733  
..         ...              ...        ...  
248  31.628000        31.719999  -0.002900  
249  31.309999        31.628000  -0.010054  
250  30.239000        31.309999  -0.034206  
251  29.830000        30.239000  -0.013526  
252  29.540001        29.830000  -0.009722  

[253 rows x 10 columns]

In [15]:
def make_dependent(x,threshold):
    if x < -threshold:
        x = -1
    elif x > threshold:
        x = 1
    else:
        x = 0
    return x

In [16]:
stock_information["dependent"] = stock_information["dependent"].apply(lambda x: make_dependent(x,0.01))

In [21]:
class USTradingHolidaysCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday(
            'NewYearsDay',
            month=1,
            day=1,
            observance=nearest_workday
        ),
        USMartinLutherKingJr,
        USPresidentsDay,
        GoodFriday,
        USMemorialDay,
        Holiday(
            'Juneteenth National Independence Day',
            month=6,
            day=19,
            start_date='2021-06-18',
            observance=nearest_workday,
        ),
        Holiday(
            'USIndependenceDay',
            month=7,
            day=4,
            observance=nearest_workday
        ),
        USLaborDay,
        USThanksgivingDay,
        Holiday(
            'Christmas',
            month=12,
            day=25,
            observance=nearest_workday
        ),
    ]
cal = USTradingHolidaysCalendar()
holidays = cal.holidays(start='2021-10-11', end='2022-10-12')

In [18]:
def opening_hours(tweet_date):
    tweet_date = pd.to_datetime(tweet_date)
    open_NYSE = tweet_date.replace(hour=14, minute=30, second=0, microsecond=0)
    close_NYSE = tweet_date.replace(hour=21, minute=0, second=0, microsecond=0)
    # if the tweet is posted before the opening hours of the stock market, it is assigned to the next day
    if (tweet_date < open_NYSE):
        dependent_date = tweet_date.date()
    # else when the tweet is posted after the opening hours, it is assigned to the next day
    else:
        dependent_date = tweet_date.date()
        dependent_date = dependent_date + datetime.timedelta(days = 1)
    # if the tweet is posted on a saturday, it is assigned to the next monday
    if (dependent_date.isoweekday() == 6):
        dependent_date = dependent_date + datetime.timedelta(days = 2)
    # if the tweet is posted on a sunday, it is assigned to the next monday 
    if (dependent_date.isoweekday() == 7):
        dependent_date = dependent_date + datetime.timedelta(days = 1)
    return dependent_date

In [ ]:
tweets["date"] = tweets["created_at"].apply(lambda x: opening_hours(x))

In [ ]:
def check_holidays(tweet_date, holidays):
    tweet_date = pd.to_datetime(tweet_date)
    # if the tweet is placed on a holiday, the tweet will be assigned to the next day that's not in the weekend
    if(tweet_date in holidays):
        tweet_date = tweet_date + datetime.timedelta(days = 1)
        if(tweet_date.isoweekday() == 6):
            tweet_date = tweet_date + datetime.timedelta(days = 2)
        elif(tweet_date.isoweekday() == 6):
            tweet_date = tweet_date + datetime.timedelta(days = 1)
    return tweet_date

In [ ]:
tweets["date"] = tweets["date"].apply(lambda x: check_holidays(x, holidays))

In [ ]:
basetable = pd.merge(stock_information, tweets, how = "left", on=['username', 'date'])